In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

input_dir = '../Results/Data/'

df1 = pd.read_csv(input_dir+'run_conf_0.5.tsv', sep='\t')
df2 = pd.read_csv(input_dir+'run_grad_0.5.tsv', sep='\t')
df3 = pd.read_csv(input_dir+'run_grad01_0.5.tsv', sep='\t')
df4 = pd.read_csv(input_dir+'run_gradinput_0.5.tsv', sep='\t')
df5 = pd.read_csv(input_dir+'run_gradmodel_0.5.tsv', sep='\t')
df6 = pd.read_csv(input_dir+'run_qbc_random_0.5.tsv', sep='\t')
df7 = pd.read_csv(input_dir+'run_hamming_align_0.5.tsv', sep='\t')

frames = [df1, df2, df3, df4, df5, df6, df7]
result = pd.concat(frames)

result['type'] = result['type'].replace({
    'random': 'Random',
    'gradient2_max': 'Gradient to input (max)',
    'gradient2_av': 'Gradient to input (average)',
    'gradient3_max': 'Gradient to model (max)',
    'gradient3_av': 'Gradient to model (average)',
    'gradient_confounding_max': 'Gradient conf. labels (max)',
    'gradient_confounding_av': 'Gradient conf. labels (average)',
    'gradient_01': 'Gradient 0-1 (max)',
    'gradient_0': 'Gradient 0-1 (average)',
    'gradient_max': 'Gradient on last layer (max)',
    'gradient_av': 'Gradient on last layer (average)',
    'qbc': 'QBC',
    'alignments': 'Alignments average distance',
    'hamming': 'Hamming average distance',
    'hamming_min': 'Hamming min distance'
})

result2 = result.reset_index(drop=True)
result3 = result2[result2['type'] == 'Random']

In [2]:
result4 = result2[['type', 'roc_aucs_test', 'ags_number']].groupby(by=['type', 'ags_number'], as_index=False).mean()

In [3]:
result4 = result4[result4['type'].isin(['Hamming average distance', 'Random'])]
result_H = result4[result4['type'].isin(['Hamming average distance'])]
result_R = result4[result4['type'].isin(['Random'])]

l1 = list(result_H.roc_aucs_test)

r = []
r2 = []
for l2 in l1:
    x1 = result_H[result_H.roc_aucs_test==l2].ags_number.mean()
    x2 = result_R[result_R.roc_aucs_test<=l2].ags_number.max()
    r.append((x2-x1)/x2)
    r2.append(x2-x1)

In [4]:

s1 = []
s2 = []
for l2 in l1:
    x1 = result_H[result_H.roc_aucs_test==l2].ags_number.mean()
    x2 = result_R[result_R.roc_aucs_test<=l2].ags_number.max()
    if (x2-x1)/x2==max(r):
        print(int(x1), x2, int(x2-x1), (x2-x1)/x2)
        s1 += [int(x1), int(x2)]
    if x2-x1==max(r2):
        print(int(x1), x2, int(x2-x1), (x2-x1)/x2)
        s2 += [int(x1), int(x2)]


35 54 19 0.35185185185185186
66 94 28 0.2978723404255319


In [5]:
print(result4[result4.ags_number.isin(s1)])
print()
print(result4[result4.ags_number.isin(s2)])

                          type  ags_number  roc_aucs_test
1020  Hamming average distance          35       0.848817
1039  Hamming average distance          54       0.868628
1290                    Random          35       0.817796
1309                    Random          54       0.847747

                          type  ags_number  roc_aucs_test
1051  Hamming average distance          66       0.876986
1079  Hamming average distance          94       0.878030
1321                    Random          66       0.862276
1349                    Random          94       0.876739
